# M03.02 Transform data with Spark in Azure Synapse Analytics



## Unit 1 of 7



### Introduction

Apache Spark provides a powerful platform for performing data cleansing and transformation tasks on large volumes of data. By using the Spark *dataframe* object, you can easily load data from files in a data lake and perform complex modifications. You can then save the transformed data back to the data lake for downstream processing or ingestion into a data warehouse.

Azure Synapse Analytics provides Apache Spark pools that you can use to run Spark workloads to transform data as part of a data ingestion and preparation workload. You can use natively supported notebooks to write and run code on a Spark pool to prepare data for analysis. You can then use other Azure Synapse Analytics capabilities such as SQL pools to work with the transformed data.



### Next unit: Modify and save dataframes



## Unit 2 of 7



### Modify and save dataframes

Apache Spark provides the *dataframe* object as the primary structure for working with data. You can use dataframes to query and transform data, and persist the results in a data lake. To load data into a dataframe, you use the **spark.read** function, specifying the file format, path, and optionally the schema of the data to be read. For example, the following code loads data from all .csv files in the **orders** folder into a dataframe named **order_details** and then displays the first five records.

```python
order_details = spark.read.csv('/orders/*.csv', header=True, inferSchema=True)
display(order_details.limit(5))
```



#### Transform the data structure

After loading the source data into a dataframe, you can use the dataframe object's methods and Spark functions to transform it. Typical operations on a dataframe include:

- Filtering rows and columns
- Renaming columns
- Creating new columns, often derived from existing ones
- Replacing null or other values

In the following example, the code uses the `split` function to separate the values in the **CustomerName** column into two new columns named **FirstName** and **LastName**. Then it uses the `drop` method to delete the original **CustomerName** column.

```python
from pyspark.sql.functions import split, col

# Create the new FirstName and LastName fields
transformed_df = order_details.withColumn("FirstName", split(col("CustomerName"), " ").getItem(0)).withColumn("LastName", split(col("CustomerName"), " ").getItem(1))

# Remove the CustomerName field
transformed_df = transformed_df.drop("CustomerName")

display(transformed_df.limit(5))
```

You can use the full power of the Spark SQL library to transform the data by filtering rows, deriving, removing, renaming columns, and any applying other required data modifications.



#### Save the transformed data

After your dataFrame is in the required structure, you can save the results to a supported format in your data lake.

The following code example saves the dataFrame into a *parquet* file in the data lake, replacing any existing file of the same name.

```python
transformed_df.write.mode("overwrite").parquet('/transformed_data/orders.parquet')
print ("Transformed data saved!")
```

> **Note:** The Parquet format is typically preferred for data files that you will use for further analysis or ingestion into an analytical store. Parquet is a very efficient format that is supported by most large scale data analytics systems. In fact, sometimes your data transformation requirement may simply be to convert data from another format (such as CSV) to Parquet!



### Next unit: Partition data files



## Unit 3 of 7



### Partition data files

Partitioning is an optimization technique that enables spark to maximize performance across the worker nodes. More performance gains can be achieved when filtering data in queries by eliminating unnecessary disk IO.



#### Partition the output file

To save a dataframe as a partitioned set of files, use the **partitionBy** method when writing the data.

The following example creates a derived **Year** field. Then uses it to partition the data.

```python
from pyspark.sql.functions import year, col

# Load source data
df = spark.read.csv('/orders/*.csv', header=True, inferSchema=True)

# Add Year column
dated_df = df.withColumn("Year", year(col("OrderDate")))

# Partition by year
dated_df.write.partitionBy("Year").mode("overwrite").parquet("/data")
```

The folder names generated when partitioning a dataframe include the partitioning column name and value in a **column=value** format, as shown here:

![Diagram representing a partitioned file folder structure.](https://learn.microsoft.com/en-us/training/wwl-data-ai/transform-data-spark-azure-synapse-analytics/media/3-partition-data-files.png)

> **Note:** You can partition the data by multiple columns, which results in a hierarchy of folders for each partitioning key. For example, you could partition the order in the example by year and month, so that the folder hierarchy includes a folder for each year value, which in turn contains a subfolder for each month value.



#### Filter parquet files in a query

When reading data from parquet files into a dataframe, you have the ability to pull data from any folder within the hierarchical folders. This filtering process is done with the use of explicit values and wildcards against the partitioned fields.

In the following example, the following code will pull the sales orders, which were placed in 2020.

```python
orders_2020 = spark.read.parquet('/partitioned_data/Year=2020')
display(orders_2020.limit(5))
```

> **Note:** The partitioning columns specified in the file path are omitted in the resulting dataframe. The results produced by the example query would not include a **Year** column - all rows would be from 2020.



### Next unit: Transform data with SQL



## Unit 4 of 7



### Transform data with SQL

The SparkSQL library, which provides the dataframe structure also enables you to use SQL as a way of working with data. With this approach, You can query and transform data in dataframes by using SQL queries, and persist the results as tables.

> **Note:** Tables are metadata abstractions over files. The data is not stored in a relational table, but the table provides a relational layer over files in the data lake.



#### Define tables and views

Table definitions in Spark are stored in the *metastore*, a metadata layer that encapsulates relational abstractions over files. *External* tables are relational tables in the metastore that reference files in a data lake location that you specify. You can access this data by querying the table or by reading the files directly from the data lake.

> **Note:** External tables are "loosely bound" to the underlying files and deleting the table *does not* delete the files. This allows you to use Spark to do the heavy lifting of transformation then persist the data in the lake. After this is done you can drop the table and downstream processes can access these optimized structures. You can also define *managed* tables, for which the underlying data files are stored in an internally managed storage location associated with the metastore. Managed tables are "tightly-bound" to the files, and dropping a managed table deletes the associated files.

The following code example saves a dataframe (loaded from CSV files) as an external table name **sales_orders**. The files are stored in the **/sales_orders_table** folder in the data lake.

```python
order_details.write.saveAsTable('sales_orders', format='parquet', mode='overwrite', path='/sales_orders_table')
```



#### Use SQL to query and transform the data

After defining a table, you can use of SQL to query and transform its data. The following code creates two new derived columns named **Year** and **Month** and then creates a new table *transformed_orders* with the new derived columns added.

```python
# Create derived columns
sql_transform = spark.sql("SELECT *, YEAR(OrderDate) AS Year, MONTH(OrderDate) AS Month FROM sales_orders")

# Save the results
sql_transform.write.partitionBy("Year","Month").saveAsTable('transformed_orders', format='parquet', mode='overwrite', path='/transformed_orders_table')
```

The data files for the new table are stored in a hierarchy of folders with the format of **Year=\*NNNN\* / Month=\*N\***, with each folder containing a parquet file for the corresponding orders by year and month.



#### Query the metastore

Because this new table was created in the metastore, you can use SQL to query it directly with the `%%sql` magic key in the first line to indicate that the SQL syntax will be used as shown in the following script:

```sql
%%sql

SELECT * FROM transformed_orders
WHERE Year = 2021
    AND Month = 1
```



#### Drop tables

When working with external tables, you can use the `DROP` command to delete the table definitions from the metastore without affecting the files in the data lake. This approach enables you to clean up the metastore after using SQL to transform the data, while making the transformed data files available to downstream data analysis and ingestion processes.

```sql
%%sql

DROP TABLE transformed_orders;
DROP TABLE sales_orders;
```



### Next unit: Exercise: Transform data with Spark in Azure Synapse Analytics



## Unit 5 of 7



### Exercise: Transform data with Spark in Azure Synapse Analytics

Now it's your chance to use Spark to transform data for yourself. In this exercise, you’ll use a Spark notebook in Azure Synapse Analytics to transform data in files.

> **Note:** To complete this lab, you will need an [Azure subscription](https://azure.microsoft.com/free) in which you have administrative access.

Launch the exercise and follow the instructions.

[Launch Exercise](https://aka.ms/mslearn-transform-data-with-spark)



### Next unit: Knowledge check



## Unit 6 of 7



### Knowledge check

1. Which method of the Dataframe object is used to save a dataframe as a file?

    - ☐ toFile()
    - ☑ write()
    > That's correct. The write() method to save a DataFrame as a file.
    - ☐ save()

2. Which method is used to split the data across folders when saving a dataframe?

    - ☐ splitBy()
    - ☐ distributeBy()
    - ☑ partitionBy()
    > That's correct. This method is an expressive and convenient operation in PySpark, which partitions the DataFrame based on given columns and distributes the partitions across the nodes in the cluster.

3. What happens if you drop an external table that is based on existing files?

    - ☐ An error – you must delete the files first
    - ☑ The table is dropped from the metastore but the files remain unaffected
    > That's correct. The tables are loosely coupled from the files allowing the table to be dropped while the files remain.
    - ☐ The table is dropped from the metastore and the files are deleted



### Next unit: Summary



## Unit 7 of 7



### Summary

In this module, you've learned how to use Apache Spark to transform data in Azure Synapse Analytics. Using Spark for complex data transformations is a common technique because of the inherent scalability of the Spark platform. You can use code in notebooks to experiment with data transformations, and then include those notebooks in automated pipelines as part of a data integration solution.

> **Tip:** Learn more about using the [Spark SQL and DataFrames Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html).



### All units complete

